## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-30-23-46-02 +0000,nyt,"Masashi Ozaki, Japan’s Dominant Golfer Known a...",https://www.nytimes.com/2025/12/30/sports/golf...
1,2025-12-30-23-44-18 +0000,nypost,California needs more snow accumulation to bol...,https://nypost.com/2025/12/30/us-news/californ...
2,2025-12-30-23-38-32 +0000,latimes,Recent storms boosted California's snowpack. B...,https://www.latimes.com/environment/story/2025...
3,2025-12-30-23-27-06 +0000,nypost,Protests erupt in Iran over nation’s tanking e...,https://nypost.com/2025/12/30/world-news/prote...
4,2025-12-30-23-26-00 +0000,wsj,South Korea Inflation Eases in December; Avera...,https://www.wsj.com/economy/south-korea-inflat...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2389/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,27
260,ukraine,11
321,new,11
7,dies,10
8,california,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
244,2025-12-30-03-22-12 +0000,bbc,'No worries': Trump downplays Chinese military...,https://www.bbc.com/news/articles/cdxwxkeg9w6o...,65
8,2025-12-30-23-13-27 +0000,nypost,Trump admin halts all childcare payments to Mi...,https://nypost.com/2025/12/30/us-news/trump-ad...,65
86,2025-12-30-20-28-24 +0000,nypost,Ukraine desperate for Trump visit — hopes it w...,https://nypost.com/2025/12/30/world-news/ukrai...,55
102,2025-12-30-19-21-43 +0000,nyt,How Russia and Ukraine Are Fighting to Shape T...,https://www.nytimes.com/2025/12/30/world/europ...,54
233,2025-12-30-04-18-08 +0000,nypost,Trump admin pausing some federal funding to Mi...,https://nypost.com/2025/12/29/us-news/trump-ad...,49


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
244,65,2025-12-30-03-22-12 +0000,bbc,'No worries': Trump downplays Chinese military...,https://www.bbc.com/news/articles/cdxwxkeg9w6o...
181,44,2025-12-30-12-00-00 +0000,nypost,Minnesota Gov. Tim Walz was warned of rampant ...,https://nypost.com/2025/12/30/us-news/minnesot...
228,41,2025-12-30-05-14-28 +0000,bbc,Bangladesh's first female prime minister Khale...,https://www.bbc.com/news/articles/cm2v256vdkyo...
59,40,2025-12-30-21-33-25 +0000,nypost,Distraught Jack Schlossberg seen with nephew e...,https://nypost.com/2025/12/30/us-news/devastat...
143,39,2025-12-30-16-55-56 +0000,nypost,"Meta snaps up AI startup Manus for $2B, drawin...",https://nypost.com/2025/12/30/business/meta-sn...
100,35,2025-12-30-19-35-00 +0000,wsj,Some Federal Reserve officials said they were ...,https://www.wsj.com/economy/central-banking/fe...
271,32,2025-12-30-00-12-41 +0000,bbc,Russia's losses in Ukraine rise faster than ev...,https://www.bbc.com/news/articles/c62n922dnw7o...
137,32,2025-12-30-17-13-49 +0000,nypost,DC pipe bomb suspect Brian Cole Jr. is autisti...,https://nypost.com/2025/12/30/us-news/dc-pipe-...
151,30,2025-12-30-16-31-18 +0000,bbc,Protests spread across Iran for third day afte...,https://www.bbc.com/news/articles/cqj2llkjv8vo...
16,30,2025-12-30-23-03-51 +0000,nyt,3 Hikers Are Found Dead on Mt. Baldy in Southe...,https://www.nytimes.com/2025/12/30/us/hikers-d...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
